### Getting Input Data
1. Use sc.textFile() to read the input text data "500input.txt", then tranform it to the RDD type data 

In [1]:
#Input
text = sc.textFile('500input.txt')

### Mapping 1
1. Mapping the RDD data into several key-value pairs (j, (M, i, value)) if the data line belong to Matrix M, otherwise, (j, (N, k, value)) if the data line belong to Matrix N. 
3. Note that Matrix_M is i * j matrix and Matrix_N is j * k matrix, value means their index value.  
4. After step 1 and 2, using the filter() to split the RDD from map1_rdd to map1_M_rdd and map1_N_rdd depending on Matrix name

In [2]:
#Mapper1
def mapper1(x):
    y = x.split(',')
    if y[0] == 'M':
        return y[2],(y[0],y[1],y[3])
    else:
        return y[1],(y[0],y[2],y[3])
map1_rdd = text.map(mapper1).sortBy(lambda x:x[0])
map1_M_rdd = map1_rdd.filter(lambda x: x[1][0]=='M')
map1_N_rdd = map1_rdd.filter(lambda x: x[1][0]=='N')

### Mapping 2
1. After mapping_1 operation, to get all possible combination, joinning the map1_M_rdd and map1_N_rdd first, each line will be ((j, (M, i, value)), (j, (N, k, value))).
2. Then, I map the new RDD data into map2_rdd where the key-value pairs will be ( ( i, k ), ( M[i][j] * N[j][k] ) ).

In [3]:
#Mapper2
join_rdd = map1_M_rdd.join(map1_N_rdd)
map2_rdd = join_rdd.map(lambda x: ((x[1][0][1],x[1][1][1]),int(x[1][0][2])*int(x[1][1][2])))

### Reducing
1. Finally, with the key (i,k) and the value, we can easily use the reduce function to get the sum of all value with identical key. 

In [4]:
#Reducing
def reducer(x,y):
    return x+y

reduce_rdd = map2_rdd.reduceByKey(reducer).sortBy(lambda x:(int(x[0][0]),int(x[0][1])))

### Output The Result
1. Using the open() to open the output traget file, and then using write() to output the data into the file
2. After finishing, closing the file to avoid putting program in the garbage collectors hands.

In [5]:
#output result
out_file = open("Outputfile.txt",'w')
for item in reduce_rdd.collect():
    out_file.write(str(item[0][0])+','+str(item[0][1])+','+str(item[1])+"\n")
out_file.close()